In [1]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Load data (update this part with your dataset)
X_train = np.load(r"P:\Project\train\X_train.npy")
y_train = np.load(r"P:\Project\train\y_train.npy")

In [17]:
from collections import Counter


In [18]:
print("New class distribution:", Counter(y_train))


New class distribution: Counter({2: 2617, 3: 2493, 1: 213, 0: 34})


In [4]:
import numpy as np
from collections import Counter

# Count current class distribution
class_counts = Counter(y_train)

# Define target count (matching Class 2)
target_count = class_counts[2]  

# Get indices of class 0 and 1
class_0_indices = np.where(y_train == 0)[0]
class_1_indices = np.where(y_train == 1)[0]

# Randomly duplicate samples from class 0 and 1 until they reach the target count
oversampled_0 = np.random.choice(class_0_indices, size=target_count, replace=True)
oversampled_1 = np.random.choice(class_1_indices, size=target_count, replace=True)

# Combine original data with oversampled data
X_train_oversampled = np.concatenate([X_train, X_train[oversampled_0], X_train[oversampled_1]])
y_train_oversampled = np.concatenate([y_train, y_train[oversampled_0], y_train[oversampled_1]])

# Shuffle the dataset
shuffle_indices = np.random.permutation(len(y_train_oversampled))
X_train_oversampled = X_train_oversampled[shuffle_indices]
y_train_oversampled = y_train_oversampled[shuffle_indices]

# Print new class distribution
print("New class distribution:", Counter(y_train_oversampled))


New class distribution: Counter({1: 2830, 0: 2651, 2: 2617, 3: 2493})


In [5]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train.flatten()
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Computed Class Weights:", class_weights_dict)

Computed Class Weights: {0: 39.38970588235294, 1: 6.287558685446009, 2: 0.5117500955292319, 3: 0.537204171680706}


In [5]:
# Train-validation split (80-20)
x_train, x_val, y_train, y_val = train_test_split(X_train_oversampled, y_train_oversampled, test_size=0.2, random_state=42, stratify=y_train_oversampled)

# Define the model-building function
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(
        units=hp.Int("num_units", min_value=128, max_value=512, step=128),
        return_sequences=False,
        input_shape=(50, 2048)
    ))
    model.add(tf.keras.layers.Dropout(hp.Float("dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(4, activation="softmax"))

    optimizer = hp.Choice("optimizer", ["adam", "rmsprop"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Define the tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=10,  # Number of different models to try
    executions_per_trial=1,  # Number of times to train each model
    directory="new_hyperparam_tuning",
    project_name="lstm_tuning"
)


Reloading Tuner from new_hyperparam_tuning\lstm_tuning\tuner0.json


In [ ]:

# Perform hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val), verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: Units={best_hps.get('num_units')}, Dropout=0.15, Optimizer={best_hps.get('optimizer')}")

Trial 10 Complete [00h 12m 11s]
val_accuracy: 0.7621519565582275

Best val_accuracy So Far: 0.7649834752082825
Total elapsed time: 01h 09m 23s
Best Hyperparameters: Units=384, Dropout=0.1, Optimizer=adam


In [7]:
# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val), verbose=1)

Epoch 1/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 52s 192ms/step - accuracy: 0.4459 - loss: 1.2363 - val_accuracy: 0.6092 - val_loss: 0.8372
Epoch 2/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 50s 187ms/step - accuracy: 0.6056 - loss: 0.8619 - val_accuracy: 0.6734 - val_loss: 0.7347
Epoch 3/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 187ms/step - accuracy: 0.6701 - loss: 0.7377 - val_accuracy: 0.7013 - val_loss: 0.6914
Epoch 4/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 185ms/step - accuracy: 0.6917 - loss: 0.6832 - val_accuracy: 0.6357 - val_loss: 0.7885
Epoch 5/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 185ms/step - accuracy: 0.6896 - loss: 0.6819 - val_accuracy: 0.6626 - val_loss: 0.6834
Epoch 6/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 185ms/step - accuracy: 0.7276 - loss: 0.6040 - val_accuracy: 0.7220 - val_loss: 0.5964
Epoch 7/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 187ms/step - accuracy: 0.7436 - loss: 0.5777 - val_accuracy: 0.7272 - val_loss: 0.5739
Epoch 8/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 49s 186ms/step - accuracy: 0.7473 - loss: 0

In [8]:
from sklearn.metrics import classification_report

# Get model predictions
y_pred = best_model.predict(x_val)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels

# Print classification report
print(classification_report(y_val, y_pred_classes))


67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       530
           1       0.84      0.99      0.91       566
           2       0.59      0.65      0.62       524
           3       0.66      0.45      0.54       499

    accuracy                           0.78      2119
   macro avg       0.77      0.77      0.77      2119
weighted avg       0.78      0.78      0.77      2119



In [9]:

# Save the best model
best_model.save("best_lstm_model.h5")